In [1]:
import numpy as np
from tnestmodel.datasets import datasets
from tnestmodel.temp_wl import compute_d_rounds
from tnestmodel.temp_fast_graph import SparseTempFastGraph
from nestmodel.graph_properties import number_of_flips_possible
import matplotlib.pyplot as plt

In [2]:
use_range_times = False

In [3]:
def switch_in_out(E):
    out = np.empty_like(E)
    out[:,0] = E[:,1]
    out[:,1] = E[:,0]
    out[:,2] = E[:,2]
    return out

In [4]:
#%load_ext snakeviz

In [5]:
#%%snakeviz --new-tab
Gs = []
time_deltas = []
hs = []
for dataset in datasets:
    print()
    print(dataset.name)
    E = dataset.read_edges()
    if dataset.is_directed:
        E = switch_in_out(E)

    nodes = np.unique(E[:, :2].ravel())
    total_time_diff = np.max(E[:,2])-np.min(E[:,2])
    time_deltas.append(total_time_diff)
    h2 = int(round(total_time_diff*1.2))

    G = SparseTempFastGraph.from_temporal_edges(E, dataset.is_directed)
    if use_range_times:
        G.times = np.arange(len(G.times))
        h2 = int(round(len(G.times*1.2)))
    G.assign_colors_to_slices(h=h2)
        
    hs.append(h2)
    Gs.append(G)
    #break



opsahl
max_d 6

email-eu2
max_d 5

email-eu3
max_d 4

dnc
max_d 5

highschool_2011
max_d 5

hospital_ward
max_d 4

ht09
max_d 5

workplace_2013
max_d 5


In [6]:
#raise ValueError()

In [7]:
from numpy.testing import assert_array_equal

In [8]:
#for G in Gs:
#    for d_iter in range(len(G.slices[0].base_partitions)):
#        G.s.reset_colors(d_iter, G.h, mode="global")
#        print(d_iter)
#        for t, G_t in zip(G.times,G.slices):
#            G.s.advance_time(t)
#            #print(G.s.current_colors[G_t.global_nodes], G_t.base_partitions[d_iter])
#            assert_array_equal(G.s.current_colors[G_t.global_nodes], G_t.base_partitions[d_iter])


In [9]:
from numba.typed import Dict
from numba import njit


In [10]:
from nestmodel.graph_properties import _source_only_number_of_flips_possible, _normal_number_of_flips_possible

@njit(cache=True)
def update_color_counts(affected_nodes, last_colors, new_colors, number_of_nodes_per_color):
    for node in affected_nodes:
        old_color = last_colors[node]
        new_color = new_colors[node]
        if old_color==new_color:
            continue
        # remove old color from count
        if number_of_nodes_per_color[old_color] == 1:
            del number_of_nodes_per_color[old_color]
        else:
            number_of_nodes_per_color[old_color]-=1
        # add new color to count
        if new_color in number_of_nodes_per_color:
            number_of_nodes_per_color[new_color]+=1
        else:
            number_of_nodes_per_color[new_color]=1
        last_colors[node] = new_color
    


class NumberOfFlipsCalculator():
    def __init__(self, G_temp, h, s = None):
        self.G_temp = G_temp
        self.h = h
        if s is None:
            self.struct = G_temp.s#get_temporal_wl_struct(h=h, base_edges=True)
        else:
            self.struct = s
        if G_temp.is_directed:
            self.last_colors = np.zeros(G_temp.num_nodes, dtype=np.int64) # assumes degree zero nodes have color zero!
            self.number_of_nodes_per_color = Dict()
            self.number_of_nodes_per_color[0] = G_temp.num_nodes
        else:
            self.last_colors = None
            self.number_of_nodes_per_color = None
        
        
        self.d = None
            
    def prepare(self, d):
        self.d = d
        self.struct.reset_colors(d=d, mode="global")
        if self.G_temp.is_directed:
            self.last_colors[:]=0
            self.number_of_nodes_per_color.clear()
            self.number_of_nodes_per_color[0] = self.G_temp.num_nodes
        
    def calc_for_slice(self, G, t):
        if self.G_temp.is_directed:
            affected_nodes = self.struct.advance_time(t)
            #print(G.global_edges, affected_nodes)
            affected_nodes = np.fromiter(affected_nodes, count = len(affected_nodes), dtype=np.int64)
            #print("current_colors", self.struct.current_colors[G.global_nodes])
            #print("set     colors", G.base_partitions[self.d])
            assert_array_equal(self.struct.current_colors[G.global_nodes], G.base_partitions[self.d])
            update_color_counts(affected_nodes, self.last_colors, self.struct.current_colors, self.number_of_nodes_per_color)
            return _source_only_number_of_flips_possible(G, self.d, self.number_of_nodes_per_color)
        else:
            return _normal_number_of_flips_possible(G, d)

In [47]:
from collections import defaultdict, Counter

def add_nodes_to_maps(u,v, basic, counter, is_directed):
    if is_directed:
        basic[u].add(v)
        counter[u][v]+=1
    else:
        basic[u].add(v)
        basic[v].add(u)
        counter[u][v]+=1
        counter[v][u]+=1

def remove_nodes_from_maps(u,v,basic, counter, is_directed):
    if is_directed:
        counter[u][v]-=1
        if counter[u][v]==0:
            basic[u].remove(v)
    else:
        counter[u][v]-=1
        if counter[u][v]==0:
            basic[u].remove(v)
        counter[v][u]-=1
        if counter[v][u]==0:
            basic[v].remove(u)    
    

class NumberOfTrianglesCalculator():
    def __init__(self, G_temp, h=None):
        self.G_temp = G_temp
        self.edges = G_temp.to_temporal_edges()
        if not h is None:
            raise NotImplementedError()
        self.count=0
        
            
    def prepare(self):
        self.future_nodes = defaultdict(set)
        self.future_nodes_count = defaultdict(Counter)
        self.past_nodes = defaultdict(set)
        self.past_nodes_count = defaultdict(Counter)
        for u,v,t in self.edges:
            add_nodes_to_maps(u,v, self.future_nodes, self.future_nodes_count, self.G_temp.is_directed)
        
    def calc_for_slice(self, G, t):
        count = 0
        for u,v in G.global_edges:
            add_nodes_to_maps(v,u, self.past_nodes, self.past_nodes_count, self.G_temp.is_directed)
            # print(u,v)
            # print(self.past_nodes[u], self.future_nodes[v])
            possible_third_nodes = self.past_nodes[u].intersection(self.future_nodes[v])
            for third_node in possible_third_nodes:
                count+=self.past_nodes_count[u][third_node]*self.future_nodes_count[v][third_node]
            remove_nodes_from_maps(u,v, self.future_nodes, self.future_nodes_count, self.G_temp.is_directed)
        return count

In [176]:
from itertools import chain
class EdgePersistenceCalculator():
    def __init__(self, G_temp, h=None):
        self.G_temp = G_temp
        self.edges = G_temp.to_temporal_edges()
        if not h is None:
            raise NotImplementedError()
        self.count=0
        
            
    def prepare(self):
        self.previous_edges = set()
        self.previous_out_degrees = defaultdict(int)
        self.previous_in_degrees = defaultdict(int)
        
    def calc_for_slice(self, G, t):
        new_edges = set()
        out_degrees = defaultdict(int)
        in_degrees = defaultdict(int)
        for v,u in G.global_edges:
            if self.G_temp.is_directed:
                new_edges.add((u,v))
                out_degrees[u]+=1
                in_degrees[v]+=1
            else:
                new_edges.add((u,v))
                new_edges.add((v,u))
                out_degrees[u]+=1
                out_degrees[v]+=1
                in_degrees[u]+=1
                in_degrees[v]+=1
                
            
        shared_edges = self.previous_edges.intersection(new_edges)
        value=0
        if len(shared_edges)>0:
            shared_degree = defaultdict(int)
            for u,v in shared_edges:
                if self.G_temp.is_directed:
                    shared_degree[u]+=1
                else:
                    shared_degree[u]+=1
            shared_nodes = set(u for u,v in shared_edges)

            value = 0
            for node in shared_nodes:
                value += shared_degree[node] / (np.sqrt(self.previous_out_degrees[node])*np.sqrt(out_degrees[node]))
        self.previous_out_degrees = out_degrees
        self.previous_in_degrees = in_degrees
        self.previous_edges = new_edges
            
        
        
        return value

In [157]:
G = SparseTempFastGraph.from_temporal_edges(np.array([(0,1,0), (1,2,1), (1,3,1), (2,0,2), (3,0,3), (2,0,4)],dtype=int), is_directed=True)
calculator = NumberOfTrianglesCalculator(G)
calculator.prepare()
# print(calculator.future_nodes_count, calculator.future_nodes)
G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)

(array([0, 1, 2, 3, 4]), array([0, 3, 0, 0, 0]))

In [158]:
ts = []
arrs = []
for h, G in zip(hs, Gs):
    calculator = NumberOfTrianglesCalculator(G)
    calculator.prepare()
    t, arr = G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True, dtype=int)
        
    t = np.array(t, dtype=np.float64)
    t = t-t.min()
    t /= t.max()
    ts.append(t)
    arrs.append(arr)

In [177]:
ts = []
pers = []
for h, G in zip(hs, Gs):
    calculator = EdgePersistenceCalculator(G)
    calculator.prepare()
    _, arr = G.compute_for_each_slice(calculator.calc_for_slice, min_size=1, call_with_time=True)
    print(arr.sum())
    pers.append(arr.sum())

1958.238135729482
2807.110314831705
725.0545856087615
716.9380535381002
31075.337368918532
29310.286495790784
18540.51996215101
11556.261380665277


In [57]:
def number_of_edges(G):
    return len(G.edges)

list_num_edges = []
for G in Gs:
    t, arr = G.compute_for_each_slice(number_of_edges, min_size=1, call_with_time=False)
    list_num_edges.append(arr.sum())

In [65]:
rows = []
for arr, num_edges, dataset in zip(arrs, list_num_edges, datasets):
    
    print(dataset.name, "\t\tnum_triangles_per_edge", np.sum(arr)/num_edges)
    rows.append((dataset.name, arr.sum(), num_edges, np.sum(arr)/num_edges))

opsahl 		num_triangles_per_edge 12.240459547142045
email-eu2 		num_triangles_per_edge 16695.927124538142
email-eu3 		num_triangles_per_edge 302.69496307360384
dnc 		num_triangles_per_edge 1082.6907365965897
highschool_2011 		num_triangles_per_edge 1745.2076807176145
hospital_ward 		num_triangles_per_edge 29717.08743523316
ht09 		num_triangles_per_edge 2097.946392544913
workplace_2013 		num_triangles_per_edge 1277.4283097588277


In [67]:
import pandas as pd

In [69]:
pd.DataFrame.from_records(rows, columns=["name", "forward_triangles", "edges", "triangles per edge"])

,name,forward_triangles,edges,triangles per edge
0,opsahl,731955,59798.0,12.240460
1,email-eu2,768179607,46010.0,16695.927125
2,email-eu3,3647777,12051.0,302.694963
3,dnc,34350529,31727.0,1082.690737
4,highschool_2011,49806482,28539.0,1745.207681
5,hospital_ward,963546843,32424.0,29717.087435
6,ht09,43675048,20818.0,2097.946393
7,workplace_2013,12553288,9827.0,1277.428310


In [70]:
import networkx as nx

In [117]:
def get_aggregated_graph(G):
    m = defaultdict(int)
    if G.is_directed:
        for u,v,t in G.to_temporal_edges():
            m[(u,v)]+=1
    else:
        for u,v,t in G.to_temporal_edges():
            m[(u,v)]+=1
            m[(v,u)]+=1
    return m


In [118]:
G.num_nodes

92

In [119]:
def count_triangles_in_aggregated(m):
    c = 0
    for (u,v), mul in m.items():
        for w in range(G.num_nodes):
            if (w,u) in m and (v,w) in m:
                c+=m[(u,v)]*m[(w,u)]*m[(v,w)]
    return c

In [166]:
nodes_per = [G.num_nodes for G in Gs]
times_per = [len(G.slices) for G in Gs]
directed_per = [G.is_directed for G in Gs]

In [127]:
counts = []
for dataset, G in zip(datasets, Gs):
    if dataset.name=="hospital_ward":
        G_out = G
    m = get_aggregated_graph(G)
    c = count_triangles_in_aggregated(m)
    print(dataset, c)
    counts.append(c)


CSVDataset(opsahl, True) 3925875
CSVDataset(email-eu2, True) 4574681847
CSVDataset(email-eu3, True) 21498570
CSVDataset(dnc, True) 203675919
CSVDataset(highschool_2011, False) 566010444
CSVDataset(hospital_ward, False) 10362849816
CSVDataset(ht09, False) 559040718
CSVDataset(workplace_2013, False) 140416356


In [144]:
559040718/6

93173453.0

In [178]:
df = pd.DataFrame.from_records(rows, columns=["name", "forward_triangles", "edges", "triangles per edge"])
df["dir"] = directed_per
df["pers"] = pers
df["all_triangles"] = np.array(counts)/6
df["nodes"] = nodes_per
df["times"] = times_per

In [179]:
df["all_edges"] = df["all_triangles"]/(df["edges"])
df["all_VT"] = df["all_triangles"]/(df["nodes"]*df["times"])
df["pers_E"] = df["pers"]/(df["edges"])
df

,name,forward_triangles,edges,triangles per edge,dir,pers,all_triangles,nodes,times,all_edges,all_VT,pers_E
0,opsahl,731955,59798.0,12.240460,True,1958.238136,6.543125e+05,1899,58911,10.942047,0.005849,0.032748
1,email-eu2,768179607,46010.0,16695.927125,True,2807.110315,7.624470e+08,162,32340,16571.331765,145.530699,0.061011
2,email-eu3,3647777,12051.0,302.694963,True,725.054586,3.583095e+06,89,8911,297.327608,4.517955,0.060166
3,dnc,34350529,31727.0,1082.690737,True,716.938054,3.394599e+07,1891,18682,1069.940004,0.960890,0.022597
4,highschool_2011,49806482,28539.0,1745.207681,False,31075.337369,9.433507e+07,126,5609,3305.479309,133.480311,1.088873
5,hospital_ward,963546843,32424.0,29717.087435,False,29310.286496,1.727142e+09,75,9453,53267.383296,2436.110774,0.903969
6,ht09,43675048,20818.0,2097.946393,False,18540.519962,9.317345e+07,113,5246,4475.619800,157.175721,0.890600
7,workplace_2013,12553288,9827.0,1277.428310,False,11556.261381,2.340273e+07,92,7104,2381.472067,35.807637,1.175970


In [134]:
print(datasets[5].mapping)

{1098: 0, 1100: 1, 1105: 2, 1108: 3, 1109: 4, 1114: 5, 1115: 6, 1116: 7, 1130: 8, 1142: 9, 1144: 10, 1148: 11, 1149: 12, 1152: 13, 1157: 14, 1159: 15, 1164: 16, 1168: 17, 1179: 18, 1181: 19, 1190: 20, 1191: 21, 1193: 22, 1196: 23, 1202: 24, 1205: 25, 1207: 26, 1209: 27, 1210: 28, 1221: 29, 1232: 30, 1238: 31, 1245: 32, 1246: 33, 1260: 34, 1261: 35, 1295: 36, 1305: 37, 1307: 38, 1320: 39, 1323: 40, 1327: 41, 1332: 42, 1352: 43, 1362: 44, 1363: 45, 1365: 46, 1373: 47, 1374: 48, 1377: 49, 1378: 50, 1383: 51, 1385: 52, 1391: 53, 1393: 54, 1395: 55, 1399: 56, 1401: 57, 1416: 58, 1460: 59, 1469: 60, 1485: 61, 1525: 62, 1535: 63, 1547: 64, 1613: 65, 1625: 66, 1629: 67, 1658: 68, 1660: 69, 1671: 70, 1701: 71, 1702: 72, 1769: 73, 1784: 74}


In [ ]:
#current_colors [313 247]
#set     colors [310 247]

In [101]:
edges1 = [[2,1,2],
[3,4,2],
[3,1,1],
[4,3,2],
[0,2,3],
[1,2,2],
[1,0,3],
[2,0,3],
[1,3,1],
[0,1,3],]

In [110]:
edges2=[[5,3,1],
[1,3,3],
[5,0,1],
[5,2,2],
[1,2,3],
[2,1,4],
[3,1,3],
[3,5,1],
[5,1,1],
[2,3,1],
[3,2,1],
[2,5,2],
[2,1,3],
[5,1,4],
[1,5,1],
[0,5,1],
[1,5,4],
[1,2,4],]

In [113]:
G1 = SparseTempFastGraph.from_temporal_edges(np.array(edges2, dtype=int), is_directed=True)
m = get_aggregated_graph(G1)
c = count_triangles_in_aggregated(m)
print(c)

54


In [ ]:
fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(16,10))
fig.suptitle("number of edges")
axs = axs.ravel()
for ax, dataset, t, arr in zip(axs, datasets, ts, num_edges_per_slice):
    ax.scatter(t, arr)
    print(dataset, arr.sum())
    ax.set_title(dataset.name+" "+str(dataset.is_directed))
        #print(dataset.name)
        #print(arr[:,0].min())
plt.show()

In [ ]:
max_depth = max([len(x.slices[0].base_partitions) for x in Gs])


for d in range(max_depth):
    fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(16,10))
    fig.suptitle("number of possible dual flips")
    axs = axs.ravel()
    for ax, dataset, t, arr in zip(axs, datasets, ts, arrs):
        if arr.shape[1]>d:
            ax.plot(t, np.cumsum(arr[:,d]+1))
            ax.set_title(dataset.name)
            ax.set_yscale("log")
            #print(dataset.name)
            #print(arr[:,0].min())
    plt.show()

In [ ]:
np.max(arrs[0][:,0]-arrs[0][:,1])

In [ ]:
np.all(arrs[0][:,0]-arrs[0][:,1]>=0)

In [ ]:
np.max(arrs[0][:,1]-arrs[0][:,3])